<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/llm/robin13b_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!pip install auto-gptq
!pip install xformers

Load model

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/robin-13B-v2-GPTQ"
model_basename = "robin-13b-GPTQ-4bit-128g.no-act.order"

use_triton = False

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# Issue: https://github.com/huggingface/transformers/issues/22762
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, unk_token="<unk>",
                                                    bos_token="<s>",
                                                    eos_token="</s>")

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)


Inference

In [ ]:
# Note: check the prompt template is correct for this model.
prompt = "Tell me about AI"
prompt_template=f'''###Human: {prompt}
###Assistant:'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt_template):])

In [ ]:
prompt_template='''You act as a prompt generator. Please do the classification task like the following examples.

###Human: change wooden table into white table in the right part
###Assistant: {"part": "right", "source object": ["wooden table"], "target object": ["white table"]}

###Human: replace blue chair and red sofa into yellow table and green chair at the bottom
###Assistant: {"part": "bottom", "source object": ["blue chair", "red sofa"], "target object": ["yellow table", "green chair"]}

###Human: replace chair into sofa at the left side
###Assistant:{"part": "left", "source object": ["chair"], "target object": ["sofa"]}

###Human: replace sofa and shelf into chair and picture at the right side in the room
###Assistant:'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt_template):])